In [ ]:
import os
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Relativer Pfad zu den Dateien
file_paths = {
    "holstein_kiel": "holstein kiel.csv",
    "kiwo": "kiwo.csv",
    "umsatzdaten": "umsatzdaten_gekuerzt.csv",
    "wetter": "wetter.csv"
}

# Überprüfen, ob alle Dateien vorhanden sind
for name, path in file_paths.items():
    if not os.path.exists(path):
        print(f"Datei {path} nicht gefunden. Bitte prüfen!")
        exit()

# Dateien laden
dfs = {name: pd.read_csv(path) for name, path in file_paths.items()}

# Holstein Kiel-Daten bereinigen
holstein_kiel_raw = dfs["holstein_kiel"]
holstein_kiel_cleaned = holstein_kiel_raw["Datum;Heimspiel;;;"].str.split(";", expand=True)
holstein_kiel_cleaned.columns = ["Datum", "Heimspiel", "Extra1", "Extra2", "Extra3"]
holstein_kiel_cleaned = holstein_kiel_cleaned[["Datum", "Heimspiel"]]
holstein_kiel_cleaned["Datum"] = pd.to_datetime(holstein_kiel_cleaned["Datum"], format="%d.%m.%Y", errors="coerce")
holstein_kiel_cleaned["Heimspiel"] = holstein_kiel_cleaned["Heimspiel"].astype(float)

# Umsatzdaten bereinigen
umsatzdaten = dfs["umsatzdaten"]
umsatzdaten["Datum"] = pd.to_datetime(umsatzdaten["Datum"], errors="coerce")

# Wetterdaten vorbereiten
wetter = dfs["wetter"]
wetter["Datum"] = pd.to_datetime(wetter["Datum"], errors="coerce")

# Kieler Woche vorbereiten
kiwo = dfs["kiwo"]
kiwo["Datum"] = pd.to_datetime(kiwo["Datum"], errors="coerce")

# Daten kombinieren
combined_data = pd.merge(umsatzdaten, wetter, on="Datum", how="left")
combined_data = pd.merge(combined_data, kiwo, on="Datum", how="left")
combined_data = pd.merge(combined_data, holstein_kiel_cleaned, on="Datum", how="left")
combined_data.fillna(0, inplace=True)

# Trainings- und Validierungszeiträume definieren
train_start, train_end = "2013-07-01", "2017-07-31"
test_start, test_end = "2017-08-01", "2018-07-31"

# Trainings- und Testdatensätze erstellen
train_data = combined_data[(combined_data["Datum"] >= train_start) & (combined_data["Datum"] <= train_end)]
test_data = combined_data[(combined_data["Datum"] >= test_start) & (combined_data["Datum"] <= test_end)]

# Ergebnisse speichern
result_list = []

# Für jede Warengruppe separat vorgehen
for group in combined_data["Warengruppe"].unique():
    train_group = train_data[train_data["Warengruppe"] == group]
    test_group = test_data[test_data["Warengruppe"] == group]
    X_train = train_group.drop(columns=["Umsatz", "Datum", "Warengruppe"])
    y_train = train_group["Umsatz"]
    X_test = test_group.drop(columns=["Umsatz", "Datum", "Warengruppe"])
    y_test = test_group["Umsatz"]
    if X_train.empty or X_test.empty:
        continue
    model = LinearRegression().fit(X_train.fillna(0), y_train)
    y_pred = model.predict(X_test.fillna(0))
    result_list.extend([{
        "id": f"{row.Datum.year}{row.Datum.month:02d}{row.Datum.day:02d}{group}",
        "umsatz": pred
    } for row, pred in zip(test_group.itertuples(index=False), y_pred)])
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"Warengruppe: {group}\nMean Squared Error: {mse:.2f}\nR² Score: {r2:.2f}\n")

# Ergebnisse in DataFrame umwandeln und speichern
result_df = pd.DataFrame(result_list)
result_df.to_csv("vorhersage_ergebnisse.csv", index=False)

# Abschließende Ausgabe
print("Alle Daten wurden berücksichtigt. Ergebnisse erfolgreich in 'vorhersage_ergebnisse.csv' gespeichert.")
